<a href="https://colab.research.google.com/github/Srvand/NLP_Transformers/blob/main/Bert_SequenceClassification_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing transformers library
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 61 kB 438 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
import transformers as trfs
import tensorflow_hub as hub

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [3]:
# Importing Data from file
df = pd.read_csv('data.csv').drop(['Unnamed: 0'],axis=1)

In [4]:
df.shape

(13083, 2)

In [6]:
df.head(5)

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


In [7]:
# Finding max length of sentence from the data
df.text.str.len().max()

433

In [8]:
# Converting Categorical column into Categhory & one hot encoding to feed into model
df['category_label']=pd.Categorical(df['category'])
df['category']=df['category_label'].cat.codes

In [9]:
# Unique number of labels
df['category'].nunique()

77

In [10]:
# Setting hyperparameters for the Model
# Max length of encoded string(including special tokens such as [CLS] and [SEP]):
MAX_SEQUENCE_LENGTH = 100
# Standard BERT model with lowercase chars only:
PRETRAINED_MODEL_NAME = 'bert-base-uncased' 
# Batch size for fitting:
BATCH_SIZE = 64
# Number of epochs
EPOCHS=5
# Setting num of labels
num_labels=df['category'].nunique()

In [ ]:
# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# def get_sentence_embeding(sentences):
#     preprocessed_text = bert_preprocess(sentences)
#     return bert_encoder(preprocessed_text)['pooled_output']

In [11]:
#Load BERT tokenizer
tokenizer = trfs.BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
# Load Bert pre-trained model
bert_model = trfs.TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME,num_labels=num_labels)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Build model input&attention mask
input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='attention_mask')
#Model output
output = bert_model([input_ids, attention_mask])[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output)
# Load the Transformers BERT model as a layer in a Keras model(Inputs&Outputs)
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

In [ ]:
#Model Summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_for_sequence_classific  TFSequenceClassifie  109541453  ['input_ids[0][0]',              
 ation_2 (TFBertForSequenceClas  rOutput(loss=None,               'attention_mask[0][0]']         
 sification)                    logits=(None, 77),                                                
                                 hidden_states=None                                         

In [13]:
# Set an optimizer
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
# Compile the model
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Split the model into Train& Test Data
X_train,X_val,Y_train,Y_val = train_test_split(df.text, df.category, test_size=0.2)

In [15]:
print(X_train.shape,X_val.shape,Y_train.shape,Y_val.shape)

(10466,) (2617,) (10466,) (2617,)


In [16]:
#Below step breaks sentences/words into Tokens,add special [CLS]&[SEP]tokens,substitute tokens with ID's
def batch_encode(X,tokenizer):
    return tokenizer.batch_encode_plus(
    X,
    max_length=MAX_SEQUENCE_LENGTH, # set the length of the sequences
    add_special_tokens=True, # add [CLS] and [SEP] tokens
    return_attention_mask=True,
    return_token_type_ids=False, # not needed for this type of ML task
    pad_to_max_length=True, # add 0 pad tokens to the sequences less than max_length
    return_tensors='tf'
)
# There are few methods for Tokenizer available. Tokenizer,Tokenizer.encode,Tokenizer.encode_plus,Tokenizer.batch_encode_plus
#input_ids,toke_type_ids,attention_mask are ouputs of tokenizer
# [CLS] token added at first position and [SEP] token added at end of each sentence
# In this case we are using batch_encode_plus as sequence of sentences list to be encoded    

In [ ]:
batch_encode(X_train,tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'input_ids': <tf.Tensor: shape=(10466, 100), dtype=int32, numpy=
array([[ 101, 2129, 2146, ...,    0,    0,    0],
       [ 101, 1045, 2347, ...,    0,    0,    0],
       [ 101, 2023, 2627, ...,    0,    0,    0],
       ...,
       [ 101, 2129, 2079, ...,    0,    0,    0],
       [ 101, 2339, 2106, ...,    0,    0,    0],
       [ 101, 1045, 7039, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(10466, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
# Sample tokenizer example
s1="This is a sample sentence"
s2="This is second one"
s=[["This is a sample sentence"],["This is second one"]]
t=tokenizer(s1,s2)
print(t)
decoded = tokenizer.decode(t["input_ids"])
print(decoded)
#input_ids,toke_type_ids,attention_mask are ouputs of tokenizer
# [CLS] token added at first position and [SEP] token added at end of each sentence

{'input_ids': [101, 2023, 2003, 1037, 7099, 6251, 102, 2023, 2003, 2117, 2028, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] this is a sample sentence [SEP] this is second one [SEP]


In [ ]:
# t=tokenizer.encode(s1,s2)
# print(t)
# t=tokenizer.encode_plus(s1)
# print(t)
# decoded = tokenizer.decode(t["input_ids"])
# print(decoded)
# t=tokenizer.batch_encode_plus(['This is a sample sentence'], ['This is second one'])
# print(t["input_ids"])

In [17]:
# Encoding the train and test features
X_train = batch_encode(X_train,tokenizer)
X_val = batch_encode(X_val,tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
X_val.values()

dict_values([<tf.Tensor: shape=(2617, 100), dtype=int32, numpy=
array([[ 101, 1045, 2106, ...,    0,    0,    0],
       [ 101, 2026, 4651, ...,    0,    0,    0],
       [ 101, 2129, 2079, ...,    0,    0,    0],
       ...,
       [ 101, 2026, 2767, ...,    0,    0,    0],
       [ 101, 2129, 2146, ...,    0,    0,    0],
       [ 101, 2028, 1997, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: shape=(2617, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>])

In [20]:
model.fit(
    x=X_train.values(),
    y=Y_train,
    validation_data=(X_val.values(),Y_val),
    epochs=1,
    batch_size=64
)

164/164 [==============================] - 402s 2s/step - loss: 3.4893 - accuracy: 0.2647 - val_loss: 2.2845 - val_accuracy: 0.6358


In [ ]:
model.evaluate(X_val.values(),Y_val)

65/82 [======================>.......] - ETA: 6s - loss: 2.2867 - accuracy: 0.6346

In [21]:
test_pred=model.predict(X_val.values())

In [23]:
test_pred

array([[0.01696849, 0.00203687, 0.00392918, ..., 0.00531666, 0.01695667,
        0.00824253],
       [0.02138414, 0.00069037, 0.04289737, ..., 0.00385781, 0.00703529,
        0.01275571],
       [0.00155555, 0.04449183, 0.0059426 , ..., 0.01161704, 0.00401327,
        0.00807164],
       ...,
       [0.06089053, 0.00401692, 0.01327771, ..., 0.00602636, 0.01769177,
        0.01430617],
       [0.00264952, 0.01344517, 0.01943692, ..., 0.00714227, 0.00347628,
        0.01688815],
       [0.01566411, 0.00275115, 0.01852572, ..., 0.00450009, 0.00687951,
        0.01311223]], dtype=float32)

In [24]:
import numpy as np
prediction = np.where(test_pred>0.5, 1,0)

In [25]:
prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# import tensorflow
# tensorflow.keras.models.save_model(
#     model,
#    '/model_path/',
#     overwrite=True,
#     include_optimizer=True,
#     save_format=None,
#     signatures=None,
#     options=None,
#     save_traces=True,
# )